In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


client = MultiServerMCPClient({
    "hybridsearch": {
        "url": "http://localhost:8000/mcp", 
        "transport": "streamable_http"
    }
})
async with client.session("hybridsearch") as session:
    tools = await load_mcp_tools(session)
    print(tools)
    # call tool
    results = await session.call_tool("hybrid_search",
    {
        "query": "có ip ko",
        "k": 30 
    })
    print(results)

[StructuredTool(name='hybrid_search', description="Retrieve restaurant and dish info from the knowledge base.\nSupports:\n- Restaurant details: name, location, cuisine, opening hours\n- Dish details: name, ingredients, dietary info, cooking style\nIncreasing 'k' allows fetching more candidate results for better coverage.", args_schema={'properties': {'query': {'description': 'Restaurant or dish name, location, or type', 'maxLength': 300, 'minLength': 3, 'title': 'Query', 'type': 'string'}, 'k': {'default': 30, 'description': 'Number of top results to retrieve: 30=fast, 50=broad', 'enum': [30, 50], 'title': 'K', 'type': 'integer'}, 'cuisine': {'default': '', 'description': 'Optional cuisine type to filter results', 'maxLength': 50, 'title': 'Cuisine', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}, metadata={'_meta': {'_fastmcp': {'tags': []}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001B9C5

In [4]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

client = MultiServerMCPClient({
    "restaurant": {
        "url": "http://localhost:8000/mcp",
        "transport": "streamable_http"
    }
})

async def call_tool(name, args=None):
    async with client.session("restaurant") as session:
        result = await session.call_tool(name, args or {})
        print(f"\n=== {name} ===")
        for content in result.content:
            print(content.text)

# 2. hybrid_search
await call_tool(
    "hybrid_search", 
    {
        "req": {
            "query": "chua ngọt", "k": 10
            }
    }
)


=== hybrid_search ===
[{"line":"D0013, MÓN ĂN CHƠI, Bánh hỏi heo quay, Heo quay da giòn ăn cùng bánh hỏi và rau sống, nước mắm, Heo quay, bánh hỏi, rau sống, nước mắm chua ngọt, Đồ mặn, Heo quay và mắm chua chua ngọt, 189000, 2-3"},{"line":"D0025, MÓN GỎI, Bò trộn salad, Thịt bò áp chảo hoặc luộc tái, cắt lát mỏng, trộn cùng rau xà lách, cà chua, hành tây và sốt dầu giấm., Thịt bò, xà lách, cà chua, hành tây, dầu giấm, Đồ chua ngọt, Mùi thơm thịt bò áp chảo, thoảng mùi tiêu, hành tây sống, 160000, 2-3"},{"line":"D0026, MÓN GỎI, Gỏi mực chua cay, Mực luộc chín vừa, cắt khoanh; trộn với hành tây, cà rốt, rau răm, ớt và nước mắm pha chua ngọt., Mực tươi, hành tây, cà rốt, rau răm, ớt, nước mắm – chanh – đường., Đồ chua, cay, ngọt, Mùi cay nồng của ớt, rau răm, 160000, 2-3"},{"line":"D0028, MÓN GỎI, Gỏi dưa leo tôm khô, Dưa leo bào sợi, trộn cùng tôm khô đã ngâm mềm, hành phi, rau thơm và nước mắm pha chua ngọt., Dưa leo, tôm khô, hành phi, rau thơm, Đồ chua ngọt, Tôm khô, hành phi vàng g

In [1]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

client = MultiServerMCPClient({
    "restaurant": {
        "url": "http://localhost:8000/mcp",
        "transport": "streamable_http"
    }
})

async def call_tool(name, args=None):
    async with client.session("restaurant") as session:
        result = await session.call_tool(name, args or {})
        print(f"\n=== {name} ===")
        for content in result.content:
            print(content.text)

# 1. list tools
async with client.session("restaurant") as session:
    tools = await load_mcp_tools(session)
    print("=== Danh sách tools ===")
    for t in tools:
        print(f"- {t.name}: {t.description}")

# 4. xem_schema
tools = await call_tool("xem_schema")

# 2. hybrid_search
await call_tool(
    "hybrid_search", 
    {
        "req": {
            "query": "cay", "k": 1
            }
    }
)

# 3. tim_theo_gia
await call_tool(
    "tim_theo_gia",
    {
        "req": {
            "max_price": 150000
            }
    }
)

# 5. goi_y_mua_kem
await call_tool(
    "goi_y_mua_kem", 
    { 
        "req" : {
            "dish_id": "D001"
            }
    }
)

await call_tool(
    "feedback_bill", 
    {
    "req" : {
        "customer_id": "C001",
        "bill_id": "B001",
        "dish_id": "D001",
        "text": "Cơm ngon lắm nha có điều ớt khô ít cay"
        }
    }
)

await call_tool(
    "feedback_bill", 
    {
    "req" : {
        "customer_id": "C001",
        "bill_id": "B001",
        "dish_id": "D001",
        "text": "Món cơm này bị mặn quá, lại còn nguội nữa rất là tuyệt"
        }
    }
)

=== Danh sách tools ===
- hybrid_search: Tìm món ăn theo tên, tag hoặc ingredient
- xem_schema: Xem schema graph hiện tại
- tim_theo_gia: Tìm món ăn dưới mức giá tối đa
- feedback_bill: Khách phản hồi → chuỗi noun-phrase

=== xem_schema ===
{"nodes":[{"id":-110,"labels":["DayOfWeek"],"name":"DayOfWeek","indexes":[],"constraints":[]},{"id":-108,"labels":["Issue"],"name":"Issue","indexes":[],"constraints":["Constraint( id=26, name='issue_name_unique', type='UNIQUENESS', schema=(:Issue {name}), ownedIndex=25 )"]},{"id":-100,"labels":["Customer"],"name":"Customer","indexes":[],"constraints":["Constraint( id=4, name='cust_id', type='UNIQUENESS', schema=(:Customer {customer_id}), ownedIndex=3 )"]},{"id":-102,"labels":["Ingredient"],"name":"Ingredient","indexes":[],"constraints":["Constraint( id=8, name='ing_name', type='UNIQUENESS', schema=(:Ingredient {name}), ownedIndex=7 )"]},{"id":-105,"labels":["Feedback"],"name":"Feedback","indexes":["target_type,target_id"],"constraints":["Constraint(

In [4]:
await call_tool(
    "feedback_bill", 
    {
    "req" : {
        "customer_id": "C001",
        "bill_id": "B001",
        "dish_id": "D0041",
        "text": "Khá là tuyệt"}
    }
)


=== feedback_bill ===
[{"feedback_id":"d383173a-f5e0-4094-a1e1-dccabe2eb129","first_phrase":"Khá","last_phrase":"tuyệt"},{"feedback_id":"d383173a-f5e0-4094-a1e1-dccabe2eb129","first_phrase":"Khá","last_phrase":"tuyệt"}]


In [7]:
async def test_list_tools():
    async with client.session("restaurant") as session:
        tools = await load_mcp_tools(session)
        print("=== Danh sách tools ===")
        for t in tools:
            print(f"- {t.name}: {t.description}")

await test_list_tools()

await call_tool("hybrid_search", {
    "query": "bún chả",
    "k": 10
})

await call_tool("tim_theo_gia", {
    "max_price": 50000
})

await call_tool("xem_schema")

await call_tool("goi_y_mua_kem", {
    "dish_id": "D001"
})


=== Danh sách tools ===
- hybrid_search: Tìm món ăn theo tên, tag hoặc ingredient
- xem_schema: Xem schema graph hiện tại
- tim_theo_gia: Tìm món ăn dưới mức giá tối đa
- goi_y_mua_kem: Gợi ý món thường được mua kèm (market-basket)

=== hybrid_search ===
meta=None content=[TextContent(type='text', text="Input validation error: 'req' is a required property", annotations=None, meta=None)] structuredContent=None isError=True

=== tim_theo_gia ===
meta=None content=[TextContent(type='text', text="Input validation error: 'req' is a required property", annotations=None, meta=None)] structuredContent=None isError=True

=== xem_schema ===
meta=None content=[TextContent(type='text', text="Error calling tool 'xem_schema': Object of type Node is not JSON serializable", annotations=None, meta=None)] structuredContent=None isError=True

=== goi_y_mua_kem ===
meta=None content=[TextContent(type='text', text="Input validation error: 'req' is a required property", annotations=None, meta=None)] structu


=== hybrid_search ===
meta=None content=[TextContent(type='text', text="Input validation error: 'req' is a required property", annotations=None, meta=None)] structuredContent=None isError=True



=== tim_theo_gia ===
meta=None content=[TextContent(type='text', text="Input validation error: 'req' is a required property", annotations=None, meta=None)] structuredContent=None isError=True

=== xem_schema ===
meta=None content=[TextContent(type='text', text="Error calling tool 'xem_schema': Object of type Node is not JSON serializable", annotations=None, meta=None)] structuredContent=None isError=True



=== goi_y_mua_kem ===
meta=None content=[TextContent(type='text', text="Input validation error: 'req' is a required property", annotations=None, meta=None)] structuredContent=None isError=True


In [ ]:
async with client.session("hybridsearch") as session:
    results = await session.call_tool("hybrid_search", {
        "query": "món bún chả",
        "k": 30
    })
    print(results)


meta=None content=[TextContent(type='text', text="Error calling tool 'hybrid_search': not enough values to unpack (expected 3, got 0)", annotations=None, meta=None)] structuredContent=None isError=True


In [5]:
async with client.session("hybridsearch") as session:
    schema = await session.call_tool("xem_schema", {})
    print(schema)


meta=None content=[TextContent(type='text', text="Error calling tool 'xem_schema': Object of type Node is not JSON serializable", annotations=None, meta=None)] structuredContent=None isError=True


In [8]:
import pandas as pd
file_path="src\data\hoanghamobile_with_summary.csv"
df = pd.read_csv(file_path, nrows=1, encoding="utf-8")
all_columns = df.columns.tolist()
# Define which column will be the content
content_column = "combined_info"
# All other columns go into metadata (exclude content_column)
metadata_columns = [col for col in all_columns if col != content_column]

from langchain_community.document_loaders import CSVLoader
loader = CSVLoader(file_path,
    csv_args={
        'delimiter': ',',
        'quotechar': '"'
    },
    content_columns=content_column,
    metadata_columns=metadata_columns,
    encoding='utf-8'
)
documents = loader.load()